# Hotpl-Insta
---

## Phase1. Crawling from instagram by [instagram-crawler](https://github.com/huaying/instagram-crawler) ([ref](https://eehoeskrap.tistory.com/335))

1. Running crawler to gather informations about post(url/description/hashtags ...)
    
> `python crawler.py hashtag -t 경성광장 -o ./output.json -n 2000 --fetch_details`
    
2. Downloading Images from gathered information(output.json)

> `python download.py`



## Phase2. Pre-processing output.json
1. Reading output.json

In [5]:
import json
import numpy as np
import pandas as pd

In [6]:
# using pandas
df = pd.read_json('output.json', encoding="utf-8")

In [28]:
# handling NaN, not a number)
df["description"] = df["description"].fillna("-")
df["description"].isna().sum()

0

2. Appending new column by abstracted hastags

In [29]:
# for abstracting hashtags, we will use regualr expression (re)
import re
hashtag_regex = "#([0-9a-zA-Z가-힣]*)"
p = re.compile(hashtag_regex)

In [43]:
# appending the captured group as list
df["hashtags"] = df["description"].apply(p.findall)

In [44]:
df

,key,caption,img_url,username,location,description,hashtags
0,https://www.instagram.com/p/B4E6Xt7FnjA/,"Image may contain: people sitting, table, drin...",https://scontent-icn1-1.cdninstagram.com/vp/3a...,iridescent_boy,송리단길 SongniDan-gil,#경성광장 ☕️,[경성광장]
1,https://www.instagram.com/p/Bp_qaxslkDs/,Image may contain: one or more people,https://scontent-icn1-1.cdninstagram.com/vp/8d...,qho_____,송리단길,옥돌이 안녕! 코기 엉덩이를 만지면 기분이 좋아집니다. #경성광장 #송리단길카페,"[경성광장, 송리단길카페]"
2,https://www.instagram.com/p/BrCJKcClElu/,"Image may contain: 1 person, sitting and indoor",https://scontent-icn1-1.cdninstagram.com/vp/37...,eyey_p,송리단길,.\n나으 여니버니랑.\n다음엔 만복래까지 꼭 가꼬야.,[]
3,https://www.instagram.com/p/B1aQ3rmFeAt/,"Image may contain: coffee cup, drink, table an...",https://scontent-icn1-1.cdninstagram.com/vp/59...,lucky_yoon_,NaN,,[]
4,https://www.instagram.com/p/B34oDp_nq7K/,"Image may contain: one or more people, people ...",https://scontent-icn1-1.cdninstagram.com/vp/24...,seo__na_1,NaN,얼마만에 평일데이트인지❤️\n#송리단길#경성광장#석촌호수카페\n사진잘찍어주는 예쁜언...,"[송리단길, 경성광장, 석촌호수카페]"
...,...,...,...,...,...,...,...
2017,https://www.instagram.com/p/BlaoHYTneHd/,Image may contain: drink and indoor,https://scontent-icn1-1.cdninstagram.com/vp/67...,b_______art,NaN,너무 좋....🖤\n#최애카페등극#카페만2차#찻잔클라쓰#경성광장,"[최애카페등극, 카페만2차, 찻잔클라쓰, 경성광장]"
2018,https://www.instagram.com/p/BlYBdxoA4zu/,Image may contain: drink,https://scontent-icn1-1.cdninstagram.com/vp/ed...,bestinvestor,"Seoul, South Korea","#B\n.\n어제 오픈한 카페 경성광장.\n커피 맛있고, 음악 좋고.\n사장님이 서...","[B, 33개월, 청양띠, 딸, 슌블리, 여기서이러시면안됩니다, 슌팸, 열대야, 밤..."
2019,https://www.instagram.com/p/BlX313-Bpu8/,"Image may contain: people sitting, table and i...",https://scontent-icn1-1.cdninstagram.com/vp/69...,syunvely_mom,NaN,신랑 백숙해줬더니\n커피사준다해서 저녁먹고 동네신상카페\n핸드드립커피만 취급하는듯....,"[예쁜커피잔, 겟썸있던골목, 송리단길, 송파카페, 신상카페, 경성광장, 핸드드립, ..."
2020,https://www.instagram.com/p/BlXfvk-gnMc/,"Image may contain: drink, coffee cup and indoor",https://scontent-icn1-1.cdninstagram.com/vp/1f...,ayama_0,NaN,지방시 찾잔 하나에 감동 감탄.. 😍\n#나도여자였나봐 .\n.\n.\n.\n#경성...,"[나도여자였나봐, 경성광장, 루이보스티, 가오픈중, 개화기컨셉, 인가요, 그런느낌적..."


3. Sending images to [Google Cloud Vision API](https://cloud.google.com/vision/docs/libraries?hl=ko#client-libraries-install-python)


    1. Activating API Key - do it on your own environment variables setting
    
> `GOOGLE_APPLICATION_CREDENTIALS=e:\PythonProjects\GoogleAPIKey\My Project 23729-b0a0947591f4.json`

    2. Install google cloud vision API

> `> pip install --upgrade google-cloud-vision`


In [17]:
import io
import os

# Imports the Google Cloud client library
from google.cloud import vision
from google.cloud.vision import types

# Instantiates a client
client = vision.ImageAnnotatorClient()

# The name of the image file to annotate
file_name = "./outputpic/000006.jpg"

# Loads the image into memory
with io.open(file_name, 'rb') as image_file:
    content = image_file.read()

image = types.Image(content=content)

# Performs label detection on the image file
response = client.label_detection(image=image)
labels = response.label_annotations

print('Labels:')
for label in labels:
    print(label.description)

Labels:
Cup
Saucer
Coffee cup
Teacup
Cup
Porcelain
Tableware
Drinkware
Serveware
Dishware


In [ ]:
python crawler.py hashtag -t 더티트렁크 -o ./output.json -n 2000 --fetch_details